<a href="https://colab.research.google.com/github/Honggu12/Machine-Learning/blob/main/06_%EC%97%AD%EC%A0%84%ED%8C%8C_%EA%B5%AC%ED%98%84%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

계층 (layer)에 대한 이해
 - 하나의 계층은 하나의 일만 전문적으로 할 수 있어야 한다. (국소적 계산)
 - Fully connected 계층은 WX+B 계산만
 - ReLU 계층은 ReLU 연산만
 - 덧셈 계층은 덧셈 연산만
 - 곱셈 계층은 곱셈 연산만
 - 모든 레이어는 미분값을 역전파 한다.

In [ ]:
# 곱셈 계층 구현하기
# forward : x * y
# backward : dx = 미분값 * y, dy = 미분값 * x
# 비고 : forward 할 때 들어온 각 값들은 저장하고 있어야 한다.

class MulLayer:

  # 초기화 : 객체에서 사용할 변수를 미리 준비
  def __init__(self):
    self.x = None
    self.y = None

  # x * y 해서 리턴
  # 계산 되는 값들을 각각 저장하고 있어야 한다.
  def forward(self, x, y):
    self.x = x
    self.y = y

    out = x * y
    return out

  # dout : 뒷층에서 넘어온 미분값
  # dx : dout * y
  # dy = dout * x

  def backward(self, dout):
    dx = dout * self.y
    dy = dout * self.x

    return dx, dy

In [ ]:
apple = 100 # 사과 1개당 가격
apple_cnt = 2 # 사과 개수
tax = 1.1 # 소비세

# 계층은 2개
# (apple * apple_cnt) * tax

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# 순전파 먼저 수행
apple_price = mul_apple_layer.forward(apple, apple_cnt)
price       = mul_tax_layer.forward(apple_price, tax)

print("최종 사과 가격 : {:.0f}".format(price))

최종 사과 가격 : 220


In [ ]:
# d돈통  / d포스기
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_cnt = mul_apple_layer.backward(dapple_price)

print("apple_cnt개의 사과 가격에 대한 미분값 : {}".format(dapple_price))
print("사과 가격에 대한 미분값 : {}".format(dapple))
print("사과 개수에 대한 미분값 : {}".format(dapple_cnt))
print("소비세에 대한 미분값 : {}".format(dtax))

apple_cnt개의 사과 가격에 대한 미분값 : 1.1
사과 가격에 대한 미분값 : 2.2
사과 개수에 대한 미분값 : 110.00000000000001
소비세에 대한 미분값 : 200


In [ ]:
# 덧셈 계층 구현하기
# forward : x + y
# backward : 뒷층에서 보내진 미분값에 * 1을 곱한다
# 비고 : forward 시에 입력된 값을 가지고 있지 않아도 된다. 역전파 시에는 미분값만 필요하니까

In [ ]:
class AddLayer:

  def __init__(self):
    p
  def forward(self, x, y):
    out = x + y
    return out

  def backward(self, dout):
    
    dx = dout * 1
    dy = dout * 1

    return dx, dy


    

신경망 레이어 만들기
 - ReLU
 - Sigmoid
 - Affine 레이어 (기하학 레이어 - Fully Connected, Dense)
 - SoftMax + Loss 레이어



In [ ]:
class ReLU:
  
  # 가지고 있어야 할 정보 - 어떤 위치의 x의 원소가 0보다 작았는지
  def __init__(self):
    self.mask = None
    
  def forward(self, x):
    self.mask = (x <= 0) # 배열 x에서 어떤 값이 음수인지 마스킹 해 놓는다. - 음수인 부분만 True
    out = x.copy() # 원본 배열 복사
    out[self.mask] = 0 # 마스크를 활용해서 음수인 부분만 True

    return out
    
  # 순전파 때 음수 였던 부분을 0으로 만들었음
  # 음수였었던 인덱스를 기억하고 있다가, 미분값 전달시에 해당 인덱스를 0으로 만든다.
  def backward(self, dout):
    dout[self.mask] = 0
    dx = dout
    return dx

    

In [ ]:
import numpy as np

x = np.array([[1.0, -0.5],
              [-2.0, 3.0]])
print(x)

[[ 1.  -0.5]
 [-2.   3. ]]


In [ ]:
relu = ReLU()
relu.forward(x)

array([[1., 0.],
       [0., 3.]])

In [ ]:
# 마스크 확인
relu.mask

array([[False,  True],
       [ True, False]])

In [ ]:
dx = np.array([[-0.1, 4.0],
               [1.3, -1.1]])

relu.backward(dx)

array([[-0.1,  0. ],
       [ 0. , -1.1]])

In [ ]:
class Sigmoid:
  
  def __init__(self):
    self.out = None # sigmoid의 순전파에서 계산된 값
  def forward(self, x):
    out = 1 / (1 + np.exp(-x))
    self.out = out

    return out

  # 시그모이드 함수를 미분하면 dsigmoid = sigmoid*(1-sigmoid)
  def backward(self, dout):
    dx = dout * self.out * (1.0 - self.out)
    return dx

In [ ]:
x = np.array([0.991, 0.34, 0.56])
sigmoid = Sigmoid()
print(sigmoid.forward(x))

dout = 0.8
print(sigmoid.backward(dout))

[0.7292854  0.58419052 0.63645254]
[0.15794257 0.19432956 0.18510456]


In [ ]:
# Affine 계층의 순전파

X = np.random.rand(2)
W = np.random.rand(2, 3)
B = np.random.rand(3)

Y = np.dot(X, W) + B
print(Y)

[0.43812425 0.99734681 1.43164589]


In [ ]:
X.shape, W.shape, B.shape

((2,), (2, 3), (3,))

In [ ]:
%cd common
!unzip common.zip

[Errno 2] No such file or directory: 'common'
/content/common
Archive:  common.zip
replace functions.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: functions.py            
replace gradient.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: gradient.py             
replace layers.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: layers.py               
replace multi_layer_net.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: multi_layer_net.py      
replace multi_layer_net_extend.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: multi_layer_net_extend.py  
replace optimizer.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: optimizer.py            
replace trainer.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: trainer.py              
replace util.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: util.py                 
replace __init__.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: __init__.py             


In [ ]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict


class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성 (매우 중요!!!!!!!!!!!!!!)
        self.layers = OrderedDict()  # 추가되는 데이터의 순서가 유지가 된다.
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x): 
        for layer in self.layers.values():
            x = layer.forward(x)
        
        return x
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 : t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1 # 마지막 계층의 미분값 설정
        dout = self.lastLayer.backward(dout) # 마지막 계층에서의 미분값 전달 받기 (SoftMaxWithLoss에서 받음)
        
        layers = list(self.layers.values()) # 저장된 레이어를 불러와서 ( 여기서는 순차적인 레이어가 저장 되어 있음 )
        layers.reverse() # 뒤집음(뒤에서 부터 전달해야 하기 때문에 )
        
        # 뒤에서 부터 역전파
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [ ]:
y